# Import data

In [6]:
#disable some annoying warning
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

#plots the figures in place instead of a new window
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import pandas.io.json as json
import geopy as geo
import time as time

#from yelp.client import Client
#from yelp.oauth1_authenticator import Oauth1Authenticator

# Helper methods

In [7]:
# http://stackoverflow.com/questions/30088006/cant-figure-out-how-to-fix-the-error-in-the-following-code
# Because there is no parent JSON object, I have to read the file line by line
def load_json_line_by_line(file_path):
    # read the entire file into a python array
    f = open(file_path, 'r')
    data = f.readlines()
    f.close()
    
    # remove the trailing "\n" from each line
    data = map(lambda x: x.rstrip(), data)

    # create one big JSON array, with each line being one entry
    data_json_str = "[" + ','.join(data) + "]"

    # now, load it into pandas (and normalize it got)
    data_df = json.json_normalize(json.loads(data_json_str))
    
    return data_df

In [8]:
def make_object_tuple(x):
    if(type(x) == 'list'):
        return tuple(map(lambda ele: make_object_tuple(ele), x))
    else:
        return x

In [33]:
# Lists like [[Coffee & Tea, coffee], [Bakeries, bakeries]] 
# have to be converted to tuples like ((Coffee & Tea, coffee), (Bakeries, bakeries))
# to be hashable

# Keeping this slow version just for demo purpose
def convert_listcolumns_to_tuplecolumns_old(df, col_names):
    i = 0
    for col_name in col_names:
        for row in df.iterrows():
            i = i+1
            print(str(i/len(df)) + "%")
            row[col_name] = eval(str(row[col_name]).replace('[','(').replace(']',')'))
    return df

In [45]:
def convert_listcolumns_to_tuplecolumns(df, col_names):
    for col_name in col_names:
        df[col_name] = df.apply(lambda row: eval(str(row[col_name]).replace('[','(').replace(']',')')), axis=1)
    return df

# Load data

Load the businesses from the dataset.

The dataset is a .json file, therefor we have to use methods form pandas.io.json to load the data.

Also set the index column to the businesses name.

In [51]:
start_time = time.time()
businesses = load_json_line_by_line("C:/VA/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_business.json")
businesses.index = businesses.pop('name')
print('Data loading took ' + str(time.time() - start_time) + " seconds.")

Data loading took 22.94520616531372 seconds.


# Basic overview

Let's have a first look at the data.
We can use head() for this.

In [52]:
businesses.head()

,attributes.Accepts Credit Cards,attributes.Accepts Insurance,attributes.Ages Allowed,attributes.Alcohol,attributes.Ambience.casual,attributes.Ambience.classy,attributes.Ambience.divey,attributes.Ambience.hipster,attributes.Ambience.intimate,attributes.Ambience.romantic,...,hours.Wednesday.close,hours.Wednesday.open,latitude,longitude,neighborhoods,open,review_count,stars,state,type
name,,,,,,,,,,,,,,,,,,,,,
"Eric Goldberg, MD",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17:00,08:00,33.499313,-111.983758,[],True,9,3.5,AZ,business
Clancy's Pub,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,40.350519,-79.886930,[],True,4,3.5,PA,business
Cool Springs Golf Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,40.356896,-80.015910,[],False,5,2.5,PA,business
Verizon Wireless,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21:00,10:00,40.357620,-80.059980,[],True,3,3.5,PA,business
Emil's Lounge,True,NaN,NaN,full_bar,False,False,False,False,False,False,...,19:00,10:00,40.408735,-79.866351,[],True,11,4.5,PA,business


# Conclusion from basic overview:

* We have many columns (104 + index!)
* We have many NaN attributes in the attributes columns
* We have list columns which have to be converted to tuples

# Convert list columns to tuple columns:

In [53]:
businesses = convert_listcolumns_to_tuplecolumns(businesses, ['categories','neighborhoods'])
businesses.head()

,attributes.Accepts Credit Cards,attributes.Accepts Insurance,attributes.Ages Allowed,attributes.Alcohol,attributes.Ambience.casual,attributes.Ambience.classy,attributes.Ambience.divey,attributes.Ambience.hipster,attributes.Ambience.intimate,attributes.Ambience.romantic,...,hours.Wednesday.close,hours.Wednesday.open,latitude,longitude,neighborhoods,open,review_count,stars,state,type
name,,,,,,,,,,,,,,,,,,,,,
"Eric Goldberg, MD",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17:00,08:00,33.499313,-111.983758,(),True,9,3.5,AZ,business
Clancy's Pub,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,40.350519,-79.886930,(),True,4,3.5,PA,business
Cool Springs Golf Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,40.356896,-80.015910,(),False,5,2.5,PA,business
Verizon Wireless,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,21:00,10:00,40.357620,-80.059980,(),True,3,3.5,PA,business
Emil's Lounge,True,NaN,NaN,full_bar,False,False,False,False,False,False,...,19:00,10:00,40.408735,-79.866351,(),True,11,4.5,PA,business


# Column overview:

Because there are so many columns, let's get an overview which columns are there:

In [12]:
for col in businesses.columns:
    print(col)

attributes.Accepts Credit Cards
attributes.Accepts Insurance
attributes.Ages Allowed
attributes.Alcohol
attributes.Ambience.casual
attributes.Ambience.classy
attributes.Ambience.divey
attributes.Ambience.hipster
attributes.Ambience.intimate
attributes.Ambience.romantic
attributes.Ambience.touristy
attributes.Ambience.trendy
attributes.Ambience.upscale
attributes.Attire
attributes.BYOB
attributes.BYOB/Corkage
attributes.By Appointment Only
attributes.Caters
attributes.Coat Check
attributes.Corkage
attributes.Delivery
attributes.Dietary Restrictions.dairy-free
attributes.Dietary Restrictions.gluten-free
attributes.Dietary Restrictions.halal
attributes.Dietary Restrictions.kosher
attributes.Dietary Restrictions.soy-free
attributes.Dietary Restrictions.vegan
attributes.Dietary Restrictions.vegetarian
attributes.Dogs Allowed
attributes.Drive-Thru
attributes.Good For Dancing
attributes.Good For Groups
attributes.Good For Kids
attributes.Good For.breakfast
attributes.Good For.brunch
attribute

# Conclusion from column overview

Whoa! That's a whole bunch of columns. Can't wait to analyze them.

But first we have to think about missing data.

# Check which columns never contain NaN

In [13]:
for col in businesses.dropna(axis = 1, how = 'any', thresh = None, subset = None, inplace = False).columns:
    print(col)

business_id
categories
city
full_address
latitude
longitude
neighborhoods
open
review_count
stars
state
type


Okay, this base columns look pretty promising: Name, type, categories, location, state, review_count and stars are available for every location

# Check if columns contain interesting information

In [14]:
businesses["type"].unique()

array(['business'], dtype=object)

Okay, we can forget about the type column, this column does not add information

In [15]:
businesses = businesses.drop('type', 1)

In [39]:
businesses["neighborhoods"].unique()

array([(), 'Greentree', 'Carnegie', 'Homestead', 'Hays', 'South Side',
       'Bellevue', 'Avalon', 'Windgap', 'Crafton Heights', 'Friendship',
       'Morningside', 'Point Breeze', 'East Liberty', 'Shadyside',
       'Larimer', 'Highland Park', ('Garfield', 'Friendship'),
       'Bloomfield', 'Homewood', 'Mt. Oliver', 'Downtown', 'Allentown',
       'Carrick', 'Overbrook', 'Brookline', 'North Side', 'Etna',
       'Troy Hill', 'Oakland', 'Lawrenceville', 'Brighton Heights',
       'Observatory Hill', 'Banksville', 'Dormont',
       ('Banksville', 'Beechview'), ('Dormont', 'Banksville'), 'Beechview',
       'West End', 'Strip District', 'Garfield', 'Bon Air',
       ('Shadyside', 'Bloomfield'), ('Shadyside', 'Friendship'),
       'The Hill District', 'Starmount', 'Dilworth', 'Ballantyne',
       'South Park', 'Uptown', 'Fourth Ward', 'First Ward', 'Third Ward',
       'Arboretum', 'Myers Park', ('South End', 'Dilworth'),
       ('Dilworth', 'Myers Park'), 'Cotswold', 'South End', 'Eliz

Neighborhoods also does not look that interesting for world-wide analysis.

In [49]:
businesses = businesses.drop('neighborhoods', 1)

# Example code to get businesses with a given attribute set

In [57]:
must_contain_value_in_columns = ['attributes.Accepts Credit Cards']
businesses.dropna(axis = 0, how = 'any', thresh = None, subset = must_contain_value_in_columns, inplace = False)

,attributes.Accepts Credit Cards,attributes.Accepts Insurance,attributes.Ages Allowed,attributes.Alcohol,attributes.Ambience.casual,attributes.Ambience.classy,attributes.Ambience.divey,attributes.Ambience.hipster,attributes.Ambience.intimate,attributes.Ambience.romantic,...,hours.Wednesday.close,hours.Wednesday.open,latitude,longitude,neighborhoods,open,review_count,stars,state,type
name,,,,,,,,,,,,,,,,,,,,,
Clancy's Pub,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,40.350519,-79.886930,(),True,4,3.5,PA,business
Emil's Lounge,True,NaN,NaN,full_bar,False,False,False,False,False,False,...,19:00,10:00,40.408735,-79.866351,(),True,11,4.5,PA,business
Alexion's Bar & Grill,True,NaN,NaN,full_bar,True,False,False,False,False,False,...,NaN,NaN,40.415517,-80.067534,Greentree,True,15,4.0,PA,business
Kings Family Restaurant,True,NaN,NaN,none,NaN,NaN,NaN,NaN,NaN,NaN,...,02:00,08:00,40.387732,-80.092874,(),True,8,3.5,PA,business
Shop N'save,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,40.387732,-80.092874,(),True,3,3.5,PA,business
Rocky's Lounge,True,NaN,NaN,full_bar,False,False,False,False,False,False,...,NaN,NaN,40.396469,-80.084942,(),True,5,4.0,PA,business
Gab & Eat,False,NaN,NaN,none,True,False,True,False,False,False,...,14:30,06:00,40.396744,-80.084800,(),True,38,4.5,PA,business
Barb's Country Junction Cafe,True,NaN,NaN,none,False,False,False,False,False,False,...,NaN,NaN,40.404638,-80.089985,Carnegie,True,5,4.0,PA,business
Extended Stay America - Pittsburgh - Carnegie,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,00:00,00:00,40.417419,-80.088557,Carnegie,True,5,4.0,PA,business
